In [11]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('..')))
import nltk

In [12]:
with open('../data/corretor_artigos.txt', 'r', encoding="utf-8") as f:
  files = f.read()

In [13]:
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

### Tokenização

In [4]:
print(f"Quantidade de palavras antes da tokenização {len(files)}")

Quantidade de palavras antes da tokenização 2605046


In [14]:
lista_tokens = nltk.tokenize.word_tokenize(files)
lista_palavras = separa_palavras(lista_tokens)
print(f"Número de palavras é {len(lista_palavras)}")

Número de palavras é 403031


In [39]:
def normalizacao(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

lista_normalizada = normalizacao(lista_palavras)
print(lista_normalizada[:10])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no']


In [53]:
len(set(lista_normalizada))

18464

In [41]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

In [42]:
def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

In [43]:
frequencia = nltk.FreqDist(lista_normalizada)
total_palavras = len(lista_normalizada)

In [44]:
def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

In [45]:
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, 'r', encoding="utf-8")
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

lista_teste = cria_dados_teste('../data/corretor_palavras.txt')
lista_teste[:10]

[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm')]

In [46]:
def avaliador(testes):
    numero_palavras = len(testes)
    acertou = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    print(f"{taxa_acerto}% de {numero_palavras} palavras")

avaliador(lista_teste)

1.08% de 186 palavras


In [51]:
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras


def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras


def troca_letra(fatias):
  novas_palavras = []
  letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
  for E, D in fatias:
    for letra in letras:
      novas_palavras.append(E + letra + D[1:])
  return novas_palavras


def inverte_letra(fatias):
    novas_palavras = []
    for E, D in fatias:
      if len(D) > 1:
        novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras


def gerador_palavras(palavra):
  fatias = []
  for i in range(len(palavra)+1):
      fatias.append((palavra[:i],palavra[i:]))

  palavras_geradas = insere_letras(fatias)
  palavras_geradas += deletando_caracteres(fatias)
  palavras_geradas += troca_letra(fatias)
  palavras_geradas += inverte_letra(fatias)
  
  return palavras_geradas

palavra_exemplo = "lógiac"
palavras_geradas = gerador_palavras(palavra_exemplo)

In [52]:
avaliador(lista_teste)

76.34% de 186 palavras


In [54]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
        else:
          desconhecida += (correta not in vocabulario)
    
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    
    print(f"{taxa_acerto}% de {numero_palavras} palavras conhecida")
    print(f"{taxa_desconhecida}% de {numero_palavras} palavras desconhecida")


vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

76.34% de 186 palavras conhecida
6.99% de 186 palavras desconhecida


In [59]:
def gerador_turbinado(palavras_geradas):
  novas_palavras = []
  for palavra in palavras_geradas:
    novas_palavras += gerador_palavras(palavra)
  return novas_palavras

palavras_g = gerador_turbinado(gerador_palavras("lóiigica"))
"lógica" in palavras_g

True

In [60]:
len(palavras_g)

787396

In [72]:
def novo_corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavras_turbinado = gerador_turbinado(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavras_turbinado)
    candidatos = [palavra]
    
    for palavra in todas_palavras:
      if palavra in vocabulario:
        candidatos.append(palavra)

    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

In [71]:
novo_corretor('logic')

Quantidade de candidatos 13


'logo'

In [76]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = novo_corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1
    
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    
    print(f"{taxa_acerto}% de {numero_palavras} palavras conhecida")
    print(f"{taxa_desconhecida}% de {numero_palavras} palavras desconhecida")


vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

55.38% de 186 palavras conhecida
6.99% de 186 palavras desconhecida


In [75]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1
    
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    
    print(f"{taxa_acerto}% de {numero_palavras} palavras conhecida")
    print(f"{taxa_desconhecida}% de {numero_palavras} palavras desconhecida")


vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

76.34% de 186 palavras conhecida
6.99% de 186 palavras desconhecida
